In [16]:
import pandas as pd # data frames
import numpy as np

In [5]:
# read data
df = pd.read_csv("https://raw.githubusercontent.com/askoshiyama/mli-cohort3/master/boston.csv")
df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,T1
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


Repeated K Fold CV (5 outer, 5 inner, repeat 5 times):
RepeatedKFold repeats K-Fold n times. It can be used when one requires to run KFold n times, producing different splits in each repetition.
https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation-iterators

In [7]:
# defining vars
input_vars = ["V1", "V2", "V3", "V4", "V5", "V6", "V7",
              "V8", "V9", "V10", "V11", "V12", "V13"]
target = ["T1"]

In [12]:
X = df.drop(labels=target, axis=1)
X.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33


In [15]:
y = df.drop(labels=input_vars, axis=1)
y.head()

,T1
0,24.0
1,21.6
2,34.7
3,33.4
4,36.2


In [75]:
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR

# define the models for selection along with their hyperparameters
models_and_parameters = {
    'linReg': (LinearRegression(), {}),
    'ridge': (Ridge(), 
              {'ridge__alpha': np.linspace(0.00001, 0.8, num=20)}),
    'svr': (SVR(kernel='rbf'),
              {'svr__C': [0.01, 0.05, 0.1, 0.5, 1], 
               'svr__gamma': [0.001, 0.0001, 0.00001]}),
    'rf': (RandomForestRegressor(n_estimators=100),
              {'rf__max_depth': [5, 10, 50, 100, 200, 500]}),
    'dtr': (DecisionTreeRegressor(),
              {"dtr__min_samples_split": [10, 20, 40], 
               "dtr__max_depth": [2, 6, 8],
               "dtr__min_samples_leaf": [20, 40, 100],
               "dtr__max_leaf_nodes": [5, 20, 100]})
}

In [80]:
# based on: https://datascience.stackexchange.com/questions/13185/nested-cross-validation-and-selecting-the-best-regression-model-is-this-the-ri
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
import numpy as np

#######################################
# configuration
REPEAT = 3
SEED = None
K_FOLD_NUM = 5
METRIC = 'neg_mean_squared_error'
#######################################

# use nested cross-validation for grid search and validation
# creates 5 folds for estimating generalization error
outer_cv = KFold(n_splits=K_FOLD_NUM, random_state=SEED, shuffle=True)

# when we train on a certain fold, we use a second cross-validation
# split in order to choose hyperparameters
inner_cv = KFold(n_splits=K_FOLD_NUM, random_state=SEED, shuffle=True)

avg_scores = dict()
for name in models_and_parameters.keys():
    avg_scores[name] = dict()
    avg_scores[name]['scores'] = []


for i in range(1, REPEAT+1):
    print(f"Repetition {i}")
    for name, (model, params) in models_and_parameters.items():    
        # first scale the data, e.g. required for SVM, Ridge
        steps = [('scale', StandardScaler()), (name, model)]
        pipe = Pipeline(steps)
        
        # choose best hyperparameters for model using KFold inner_cv
        model_with_gridsearch = GridSearchCV(
            estimator=pipe, 
            param_grid=params, 
            iid=False,
            cv=inner_cv, 
            scoring=METRIC
        )        

        # estimate generalization error on the K-fold splits of the data
        # average test set scores over several dataset splits (defined by outer_cv)
        scores_across_outer_folds = cross_val_score(
            model_with_gridsearch,
            X, y.values.ravel(), 
            cv=outer_cv, 
            scoring=METRIC)
        
        avg_score_model = np.mean(scores_across_outer_folds)
        avg_scores[name]['scores'].append(avg_score_model)
        # print(f"[{i}] Model: {name}:\tAverage MSE in the outer folds: {avg_score_model}")
        # print()

print()
print("Avg scores per model over 5 repetitions:")
for name, vals in avg_scores.items():
    scores = vals['scores']
    avg_score = np.mean(scores)
    print(f"{name}:\t{avg_score}") # \tscores: {scores}
    avg_scores[name]['avg_score'] = avg_score
    
many_stars = '\n' + '*' * 20 + '\n'
print(many_stars + 'choose the best model and refit on the whole dataset' + many_stars)

print(many_stars + '\n' + str(avg_scores) + '\n' + many_stars)

best_model_name, best_model_avg_score = max(
    avg_scores.items(),
    key=(lambda name_s_as: name_s_as[1]['avg_score']))

print(f"Best model: {best_model_name}\nBest avg score: {best_model_avg_score['avg_score']}")
      
best_model, best_model_params = models_and_parameters[best_model_name]
      
# now we refit this best model on the whole dataset so that we can start
# making predictions on other data, and now we have a reliable estimate of
# this model's generalization error and we are confident this is the best model
# among the ones we have tried
steps = [('scale', StandardScaler()), (best_model_name, best_model)]
pipe = Pipeline(steps)
final_regressor = GridSearchCV(
    estimator=pipe,
    param_grid=best_model_params,
    iid=False,
    cv=inner_cv, 
    scoring=METRIC)
  
final_regressor.fit(X, y)

print('Best model: \n\t{}'.format(best_model), end='\n\n')
print('Estimation of its generalization error (negative mean squared error):\n\t{}'.format(
    best_model_avg_score), end='\n\n')
print('Best parameter choice for this model: \n\t{params}'
      '\n(according to cross-validation `{cv}` on the whole dataset).'.format(
      params=final_regressor.best_params_, cv=inner_cv))



Repetition 1
Repetition 2
Repetition 3

Avg scores per model over 5 repetitions:
linReg:	-23.969542961979965
ridge:	-23.697708622145367
svr:	-64.36737536557708
rf:	-11.160624528081229
dtr:	-22.239300946323922

********************
choose the best model and refit on the whole dataset
********************


********************

{'linReg': {'scores': [-23.435396510573558, -23.969375647409016, -24.503856727957306], 'avg_score': -23.969542961979965}, 'ridge': {'scores': [-23.2061478654615, -24.05761318931506, -23.829364811659534], 'avg_score': -23.697708622145367}, 'svr': {'scores': [-64.67703540663572, -64.14164887409714, -64.28344181599837], 'avg_score': -64.36737536557708}, 'rf': {'scores': [-11.170231632367315, -9.947204026849159, -12.364437925027207], 'avg_score': -11.160624528081229}, 'dtr': {'scores': [-21.442668473762307, -21.2351681552346, -24.04006620997486], 'avg_score': -22.239300946323922}}

********************

Best model: rf
Best avg score: -11.160624528081229


KeyError: 'best_param'